In [1]:
import numpy as np
import pandas as pd

In [10]:
package_file = 'package.csv'
product_file = 'product.csv'

product_data = pd.read_csv(product_file, sep=';', encoding='latin1')
package_data = pd.read_csv(package_file, sep=';')

# 1. Auscultation
## Etude des données du fichier 'package'

In [20]:
package_data.head()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0699-0155_60f08fe4-aacf-c815-e053-2991aa0a3ded,0699-0155,0699-0155-44,1 CANISTER in 1 CARTON (0699-0155-44) > 125 g...,19000101.0,NaN,N,N
1,0699-0311_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0311,0699-0311-16,"498 g in 1 BOTTLE, PLASTIC (0699-0311-16)",19000101.0,NaN,N,N
2,0699-0312_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0312,0699-0312-16,"498 g in 1 BOTTLE, PLASTIC (0699-0312-16)",19000101.0,NaN,N,N
3,0699-0317_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0317,0699-0317-16,"498 g in 1 BOTTLE, PLASTIC (0699-0317-16)",19000101.0,NaN,N,N
4,0699-0404_5bc1c974-e7b8-094c-e053-2991aa0ab726,0699-0404,0699-0404-01,28 g in 1 JAR (0699-0404-01),19000101.0,NaN,N,N


In [24]:
package_data.isnull().sum().sort_values()

PACKAGEDESCRIPTION         0
NDC_EXCLUDE_FLAG           0
SAMPLE_PACKAGE             0
PRODUCTID                 41
NDCPACKAGECODE           193
PRODUCTNDC               210
STARTMARKETINGDATE       408
ENDMARKETINGDATE      167431
dtype: int64

### Colonne PRODUCTID
On remarque que, pour chaque objet, la valeur du 'PRODUCTNDC' y est incluse telle un préfixe. Dans la documentation NDC, il est précisé que c'est pour prévenir le duplicata de lignes. Il y a 41 valeurs nulles.

### Colonne PACKAGEDESCRIPTION
Cette colonne est sous forme de phrase et contient de multiples informations. Le volume, son unité, le nombre de contenant et son type. S'il existe plusieurs contenants pour un objet, ils sont concaténés avec un séparateur '>' de manière hiérarchique. Aussi, on remarque aussi que la valeur du 'NDCPACKAGECODE' de l'objet est incluse entre paranthèses. 

### Colonne STARTMARKETINGDATE
Les valeurs de cette colonne sont de type date. Elles ont l'air séquentielles. 

### Colonne ENDMARKETINGDATE
Les valeurs de cette colonne sont de type date. Il existe beaucoup de valeurs manquantes.

### Colonne NDC_EXCLUDE_FLAG

In [14]:
package_data['NDC_EXCLUDE_FLAG'].unique()

array(['N'], dtype=object)

Tous les objets possèdent la valeur N et ne présente aucune valeur manquante. 

### Colonne SAMPLE_PACKAGE

In [17]:
package_data['SAMPLE_PACKAGE'].value_counts()

N    173223
Y       664
Name: SAMPLE_PACKAGE, dtype: int64

Les valeurs possibles sont 'Y' ou 'N'. Il y a une majorité de 'N' et aucune valeur manquante.

## Etude des données du fichier 'product'

In [40]:
product_data.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0699-0155_60f08fe4-aacf-c815-e053-2991aa0a3ded,0699-0155,HUMAN PRESCRIPTION DRUG,Topex Neutral Fluoride Foam,NaN,sodium fluoride,"AEROSOL, FOAM",DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,Dentsply LLC. Professional Division Trading as...,SODIUM FLUORIDE,.02,g/g,NaN,NaN,N,20201231.0
1,0699-0311_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0311,HUMAN PRESCRIPTION DRUG,Topex 60 Second Fluoride Gel,NaN,Sodium Fluoride,GEL,DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,Dentsply LLC. Professional Division Trading as...,SODIUM FLUORIDE,25.9,mg/g,NaN,NaN,N,20201231.0
2,0699-0312_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0312,HUMAN PRESCRIPTION DRUG,Topex 60 Second Fluoride Gel,NaN,Sodium Fluoride,GEL,DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,Dentsply LLC. Professional Division Trading as...,SODIUM FLUORIDE,25.9,mg/g,NaN,NaN,N,20201231.0
3,0699-0317_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0317,HUMAN PRESCRIPTION DRUG,Topex 60 Second Fluoride Gel,NaN,Sodium Fluoride,GEL,DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,Dentsply LLC. Professional Division Trading as...,SODIUM FLUORIDE,25.9,mg/g,NaN,NaN,N,20201231.0
4,0699-0404_5bc1c974-e7b8-094c-e053-2991aa0ab726,0699-0404,HUMAN PRESCRIPTION DRUG,Dry Socket,NaN,Guaiacol and Eugenol,PASTE,DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,"Dentsply LLC, Professional Division, trading a...",GUAIACOL; EUGENOL,.0416; .0416,g/g; g/g,"Standardized Chemical Allergen [EPC],Increased...",NaN,N,20201231.0


In [19]:
product_data.isnull().sum().sort_values()

PRODUCTID                               0
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
DOSAGEFORMNAME                          0
STARTMARKETINGDATE                      0
NDC_EXCLUDE_FLAG                        0
MARKETINGCATEGORYNAME                   0
NONPROPRIETARYNAME                      4
PROPRIETARYNAME                         6
LABELERNAME                           557
ROUTENAME                            1932
ACTIVE_INGRED_UNIT                   2309
ACTIVE_NUMERATOR_STRENGTH            2309
SUBSTANCENAME                        2309
LISTING_RECORD_CERTIFIED_THROUGH     4325
APPLICATIONNUMBER                   13097
PHARM_CLASSES                       51009
PROPRIETARYNAMESUFFIX               83075
DEASCHEDULE                         88815
ENDMARKETINGDATE                    88915
dtype: int64

### Colonne PRODUCTID

In [22]:
product_data['PRODUCTTYPENAME'].value_counts()

HUMAN OTC DRUG                 46172
HUMAN PRESCRIPTION DRUG        44526
NON-STANDARDIZED ALLERGENIC     2008
PLASMA DERIVATIVE                294
STANDARDIZED ALLERGENIC          124
VACCINE                          108
CELLULAR THERAPY                   6
Name: PRODUCTTYPENAME, dtype: int64

On remarque que, pour chaque objet, la valeur du 'PRODUCTNDC' y est incluse telle un préfixe. Dans la documentation NDC, il est précisé que c'est pour prévenir le duplicata de lignes.

### Colonne PRODUCTTYPENAME
Il y a 7 valeurs possibles textuelles catégorielles dans cette colonne.

### Colonne PROPRIETARYNAME

In [23]:
product_data['PROPRIETARYNAME'].nunique()

32716

In [37]:
product_data['PROPRIETARYNAME'][393:401]

393    SUGAR FREE ORIGINAL SWISS HERB COUGH SUPPRESSA...
394    Sugar Free Mountain Herb Cough Suppressant Thr...
395                                              Albuked
396                                              Demerol
397                                              Demerol
398                                              Desoxyn
399                                               OXYGEN
400                                               Oxygen
Name: PROPRIETARYNAME, dtype: object

Dans cette colonne, il existe un grand nombre de valeurs différentes. Les mêmes valeurs peuvent être présentes sous 
différentes formes notamment en minuscules ou majuscules, il existe donc une inconsistance entre les valeurs.

### Colonne PROPRIETARYNAMESUFFIX

In [34]:
product_data['PROPRIETARYNAMESUFFIX'].nunique()

4022

Dans cette colonne, il y a un nombre important de valeurs manquantes. Ces valeurs sont textuelles et il existe un nombre 
élevé de valeurs différentes.

### Colonne NONPROPRIETARYNAME

In [39]:
product_data['NONPROPRIETARYNAME'].nunique()

16257

In [36]:
product_data['NONPROPRIETARYNAME'][2:6]

2         Sodium Fluoride
3         Sodium Fluoride
4    Guaiacol and Eugenol
5         sodium fluoride
Name: NONPROPRIETARYNAME, dtype: object

Cette colonne présente seulement 4 valeurs manquantes. Ceux sont des données textuelles inconsistantes, par exemple pouvant représenter la même valeur en caratères minuscules ou majuscules. Il y a un nombre très important de valeurs différentes.

### Colonne DOSAGEFORMNAME

In [41]:
product_data['DOSAGEFORMNAME'].value_counts()

TABLET                              15442
TABLET, FILM COATED                  9387
LIQUID                               8764
CREAM                                5104
CAPSULE                              4824
                                    ...  
SOLUTION, GEL FORMING / DROPS           1
FOR SUSPENSION, EXTENDED RELEASE        1
INJECTION, LIPID COMPLEX                1
CHEWABLE GEL                            1
CRYSTAL                                 1
Name: DOSAGEFORMNAME, Length: 134, dtype: int64

Cette colonne contient 134 différentes valeurs textuelles. Comme on peut le voir, différentes catégories peuvent être affectées au même objet. La colonne ne présente aucune valeur manquante.

### Colonne ROUTENAME

In [42]:
product_data['ROUTENAME'].value_counts()

ORAL                                                                         54704
TOPICAL                                                                      21547
INTRAVENOUS                                                                   3374
INTRADERMAL; PERCUTANEOUS; SUBCUTANEOUS                                       1315
RESPIRATORY (INHALATION)                                                      1257
                                                                             ...  
INTRA-ARTICULAR; INTRALESIONAL; INTRAMUSCULAR; INTRASYNOVIAL; SOFT TISSUE        1
ORAL; ORAL; ORAL                                                                 1
NASOGASTRIC                                                                      1
HEMODIALYSIS; INTRAVENOUS                                                        1
INTRACORONARY; RESPIRATORY (INHALATION)                                          1
Name: ROUTENAME, Length: 180, dtype: int64

Cette colonne contient 180 différentes valeurs textuelles. Comme on peut le voir, différentes catégories peuvent être affectées au même objet. La colonne présente 1932 valeurs manquantes.